In [1]:
# import torch
# from torch import nn

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, LSTM, TimeDistributed, RepeatVector
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
with open('/content/drive/My Drive/Stock_Prediction/all_stock_history_price.pickle', 'rb') as f:
    all_stock_history_price = pickle.load(f)

In [4]:
all_stock_history_price_np = np.array(all_stock_history_price)

In [5]:
print(all_stock_history_price_np.shape)
all_stock_history_price[0, 4, :]

(1147, 3946, 5)


In [6]:
all_stock_history_price_np.transpose(1, 0, 2).reshape(3946, -1)

array([[   56.45,    56.65,    56.05, ...,  8143.45,  8207.85,     0.  ],
       [   56.8 ,    56.9 ,    56.15, ...,  8162.54,  8211.4 ,     0.  ],
       [   56.5 ,    57.75,    56.5 , ...,  8211.4 ,  8327.62,     0.  ],
       ...,
       [  106.  ,   106.55,   105.65, ..., 12846.42, 12877.25,     0.  ],
       [  105.7 ,   106.1 ,   105.5 , ..., 12827.4 , 12917.03,     0.  ],
       [  106.5 ,   106.65,   105.65, ..., 12875.23, 12898.82,     0.  ]])

In [7]:
def normalize(all_stock):
    all_stock_normed = []
    for one_stock in all_stock:
        one_stock_normed = []
        for col in range(one_stock.shape[-1]):
            x = one_stock[:, col]
            if np.max(x) > np.min(x):
                # Normalizing time series
                x = (x - np.mean(x)) / (np.max(x) - np.min(x))
            else:
                x.fill(0.0)           
            one_stock_normed.append(x)
        all_stock_normed.append(np.array(one_stock_normed).T)
    return np.array(all_stock_normed)


In [8]:
all_stock_history_price_np_normed = normalize(all_stock_history_price_np)

In [9]:
all_stock_history_price_np_normed.shape

(1147, 3946, 5)

In [10]:
# reshape data to (3946, 5735)
all_stock_history_price_np_normed_reshaped = all_stock_history_price_np_normed.transpose(1, 0, 2).reshape(3946, -1)

In [11]:
print(all_stock_history_price_np_normed_reshaped.shape)

(3946, 5735)


In [ ]:
# all_stock_history_price_np_normed_reshaped = all_stock_history_price_np_normed_reshaped.T

In [ ]:
# all_stock_history_price_np_normed_reshaped.shape

(5735, 3946)

In [17]:
model = Sequential()

model.add(LSTM(128, input_shape =(365, 5), return_sequences=True))
model.add(LSTM(128, return_sequences = True))
model.add(LSTM(units = 50))
model.add(Dense(5735))


model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 365, 61)           16348     
_________________________________________________________________
dropout (Dropout)            (None, 365, 61)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 365, 50)           22400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 365, 50)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 365, 50)           20200     
_________________________________________________________________
dropout_2 (Dropout)          (None, 365, 50)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 50)                2

In [18]:
all_stock_history_price_np_normed_reshaped[0:2]

array([[-0.19006433, -0.18833563, -0.19165305, ..., -0.15848929,
        -0.15567742,  0.        ],
       [-0.18426002, -0.18426596, -0.18998888, ..., -0.15547806,
        -0.15511779,  0.        ]])

In [19]:
all_stock_history_price_np_normed_reshaped.shape[0]

3946

In [ ]:
ref_day=365
predict_day=61
split_boundary = int(all_stock_history_price_np_normed_reshaped.shape[0] * 0.9)
testSequence = all_stock_history_price_np_normed_reshaped[-split_boundary:]
test_x = [testSequence[i:i + ref_day] for i in range(testSequence.shape[0] - ref_day - predict_day)]
test_y = [testSequence[i + ref_day:i + ref_day + predict_day] for i in range(testSequence.shape[0] - ref_day - predict_day)]

for round in range(100):
    for i in range(split_boundary):
        train_x = all_stock_history_price_np_normed_reshaped[i:i+ref_day]
        train_y = all_stock_history_price_np_normed_reshaped[i+ref_day:i+ref_day+predict_day]
        print(train_y.shape)

        history = model.fit(train_x, train_y, epochs = 1, shuffle=False, validation_data=(test_x, test_y))


(61, 5735)


ValueError: ignored

In [ ]:
def split_to_x_and_y(all_stock, ref_day=365, predict_day=61):
    train_x_all = []

    test_x_all = []

    train_y_all = []

    test_y_all = []
    for one_stock in all_stock[0:10]:
        X, Y = [], []
        for i in range(all_stock.shape[1] - predict_day - ref_day):
            # print(one_stock[i:+ref_day].shape)
            X.append(one_stock[i:i+ref_day])
            Y.append(one_stock[i+ref_day:i+ref_day+predict_day])

        X = np.array(X)
        # print(X.shape)

        Y = np.array(Y)
        split_boundary = int(X.shape[0] * 0.9)

        train_x = X[: split_boundary]
        test_x = X[split_boundary:]
        train_y = Y[: split_boundary]
        test_y = Y[split_boundary:]

        train_x_all.append(train_x)
        test_x_all.append(test_x)
        train_y_all.append(train_y)
        test_y_all.append(test_y)
        
    return np.array(train_x_all), np.array(test_x_all), np.array(train_y_all), np.array(test_y_all)
